# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [141]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Masking, Layer, BatchNormalization, Dense, Add, LSTM, Input, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [142]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [143]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [144]:
english, german = [], []

SOS, EOS = '<start>', '<end>'
for sentence in data_examples:         
    sentences = sentence.split('\t')
    #print(sentences)
    #sentences = sent_tokenize(preprocess_sentence(sentence))
    english.append(preprocess_sentence(sentences[0]))
    german.append(f'{SOS} {preprocess_sentence(sentences[1])} {EOS}')

english = np.array(english)
german = np.array(german)

In [145]:
def create_german_tokenizer(list_of_strings):
    
    tokenizer = Tokenizer(num_words=None, 
                          filters=" ",
                          char_level=False)
    
    tokenizer.fit_on_texts(list_of_strings)
    
    return tokenizer

In [146]:
german_tokenizer = create_german_tokenizer(german)
tokenized_german = german_tokenizer.texts_to_sequences(german)
vocab_size = len(german_tokenizer.word_index) # np.max(np.max(tokenized_german, axis=0))

In [147]:
def pad_german_sequence(sequence_chunks):
    
    padded_sequences = pad_sequences(sequence_chunks, padding='post', value=0)
    
    return np.array(padded_sequences)

In [148]:
padded_german_sequences = pad_german_sequence(tokenized_german)

In [149]:
random_indices = np.random.choice(len(english), size=5, replace=False)

for idx in random_indices:
    print(f"English sentence:\t\t{english[idx]}")
    print(f"German sentence:\t\t{german[idx]}")
    print(f"German sentence sequence:\t{tokenized_german[idx]}\n")


English sentence:		don't deceive tom .
German sentence:		<start> taeusche tom nicht . <end>
German sentence sequence:	[1, 3000, 5, 12, 3, 2]

English sentence:		i'm not satisfied .
German sentence:		<start> ich bin nicht zufrieden . <end>
German sentence sequence:	[1, 4, 15, 12, 508, 3, 2]

English sentence:		don't stand up .
German sentence:		<start> stehen sie nicht auf . <end>
German sentence sequence:	[1, 407, 8, 12, 29, 3, 2]

English sentence:		i'm a bit hungry .
German sentence:		<start> ich bin leicht hungrig . <end>
German sentence sequence:	[1, 4, 15, 479, 327, 3, 2]

English sentence:		i may know tom .
German sentence:		<start> ich koennte tom kennen . <end>
German sentence sequence:	[1, 4, 210, 5, 367, 3, 2]



## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [150]:
# Load embedding module from Tensorflow Hub

#embedding_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", 
#                                 output_shape=[128], input_shape=[], dtype=tf.string)
embedding_layer = hub.load('models/tf2-preview_nnlm-en-dim128_1')

In [151]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [152]:
x_train, x_valid, y_train, y_valid = train_test_split(english, padded_german_sequences, test_size=0.2)
print(x_train.shape)
print(y_train.shape)

(16000,)
(16000, 14)


In [153]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))

In [154]:
train_dataset.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(14,), dtype=tf.int32, name=None))

In [155]:
def map_dataset(dataset):
    
    def embed_english(image, label):        
        return embedding_layer(tf.strings.split(image,' ')), label
    
    def filter_length(image, label):
        return tf.math.less(tf.cast(tf.shape(image)[0], dtype=tf.int32), 13)
    
    def padding_english(image, label):
        padded = tf.pad(image, [[13-tf.shape(image)[0], 0], [0,0]], 'constant')
        return padded, label

    dataset = dataset.map(embed_english)
    dataset = dataset.filter(filter_length)
    dataset = dataset.map(padding_english)
    dataset = dataset.batch(16, drop_remainder=True)
 
    return dataset

In [156]:
train_dataset_mapped = map_dataset(train_dataset)
valid_dataset_mapped = map_dataset(valid_dataset)

print(train_dataset_mapped.element_spec)
print(valid_dataset_mapped.element_spec)

(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 14), dtype=tf.int32, name=None))
(TensorSpec(shape=(16, None, 128), dtype=tf.float32, name=None), TensorSpec(shape=(16, 14), dtype=tf.int32, name=None))


In [157]:
en, de = next(iter(train_dataset_mapped.take(1)))

In [158]:
#en, de = next(iter_train_dataset)
print(f"Shape of the English data example from the training Dataset: {en.shape}")
print(f"German data example Tensor from the validation Dataset: {de.shape}")

Shape of the English data example from the training Dataset: (16, 13, 128)
German data example Tensor from the validation Dataset: (16, 14)


## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [159]:
class CustomLayer(Layer):
    
    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        
        self.embedding = tf.Variable(initial_value=tf.random.uniform(shape=(128,)), trainable=True)

    def call(self, inputs):
        output = tf.tile(tf.reshape(self.embedding, shape=(1, 1, tf.shape(self.embedding)[0])),
                         [tf.shape(inputs)[0], 1, 1])                            
        return output

In [160]:
en, de = next(iter(train_dataset_mapped.take(1)))

custom_layer = CustomLayer()
test_result = custom_layer(en)

print(f'English data examples from the training Dataset: {en.shape}')
print(f'Resulting layer tensor shape: {test_result.shape}')

English data examples from the training Dataset: (16, 13, 128)
Resulting layer tensor shape: (16, 1, 128)


## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [161]:
def EncoderModel(input_shape):
    
    inputs = Input(shape=input_shape)
    h = CustomLayer()(inputs)
    h = Masking(mask_value=0)(h)
    output, hidden_states, cell_states = LSTM(512, return_state=True)(h)
    
    encoder = Model(inputs=inputs, outputs=[hidden_states, cell_states])
    
    return encoder

In [162]:
en, de = next(iter(train_dataset_mapped.take(1)))

encoder = EncoderModel(en[0].shape)
result = encoder(en)

print(encoder.summary())

print(f'Shape of resulting tensor of hidden states: {result[0].shape}')
print(f'Shape of resulting tensor of cell states: {result[1].shape}')

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        [(None, 13, 128)]         0         
_________________________________________________________________
custom_layer_31 (CustomLayer (None, 1, 128)            128       
_________________________________________________________________
masking_28 (Masking)         (None, 1, 128)            0         
_________________________________________________________________
lstm_54 (LSTM)               [(None, 512), (None, 512) 1312768   
Total params: 1,312,896
Trainable params: 1,312,896
Non-trainable params: 0
_________________________________________________________________
None
Shape of resulting tensor of hidden states: (16, 512)
Shape of resulting tensor of cell states: (16, 512)


## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [163]:
class DecoderModel(Model):
    
    def __init__(self, token_size):
        super(DecoderModel, self).__init__()
        #self.token_size = token_size
        self.embedding = Embedding(token_size, 128, mask_zero=0)
        self.lstm = LSTM(512, return_state=True, return_sequences=True)
        self.dense = Dense(token_size)
        
    def call(self, inputs, hidden_state=None, cell_state=None):
        h = self.embedding(inputs)
        if (hidden_state is None) and (cell_state is None):
            h = self.lstm(h)
        else: 
            h, hidden_state, cell_state = self.lstm(h, initial_state=(hidden_state, cell_state))
        h = self.dense(h)
        
        return h, hidden_state, cell_state

In [164]:
en, de = next(iter(train_dataset_mapped.take(1)))

encoder = EncoderModel(en[0].shape)
hidden_states, cell_states = encoder(en)
 
decoder = DecoderModel(vocab_size)
result = decoder(de, hidden_states, cell_states)

print(decoder.summary())

print(f'Shape of outputs: {result[0].shape}')
print(f'Shape of resulting tensor of hidden states: {result[1].shape}')
print(f'Shape of resulting tensor of cell states: {result[2].shape}')

Model: "decoder_model_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     multiple                  735104    
_________________________________________________________________
lstm_56 (LSTM)               multiple                  1312768   
_________________________________________________________________
dense_26 (Dense)             multiple                  2946159   
Total params: 4,994,031
Trainable params: 4,994,031
Non-trainable params: 0
_________________________________________________________________
None
Shape of outputs: (16, 14, 5743)
Shape of resulting tensor of hidden states: (16, 512)
Shape of resulting tensor of cell states: (16, 512)


## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [165]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [166]:
def decoder_data(batch_data):
    return batch_data[:, :-1], batch_data[:, 1:]

In [175]:
tf.config.experimental_run_functions_eagerly(True)
@tf.function
def grad(en, de_inputs, de_outputs):
    
    encoder = EncoderModel(en[0].shape)
    decoder = DecoderModel(vocab_size)
    
    with tf.GradientTape() as tape:
        hidden_states, cell_states = encoder(en)
        outputs, hidden_state_de, cell_states_de = decoder(de_inputs, hidden_states, cell_states)
        
        loss_value = loss(de_outputs, outputs)
        
    return outputs, loss_value, tape.gradient(loss_value, encoder.trainable_variables + decoder.trainable_variables)
    

In [176]:
def training_(num_epochs, train_dataset, valid_dataset, optimizer, loss, grad):

    train_loss_results = []
    valid_loss_results = []
    #train_accuracy_results = []
    
    for epoch in range(num_epochs):
    
        mean_loss = tf.keras.metrics.Mean()
        mean_accuracy = tf.keras.metrics.CategoricalAccuracy()

        for en, de in train_dataset:
            de_inputs, de_outputs = decoder_data(de)
        
            outputs, loss_value, grads = grad(en, de_inputs, de_outputs)
            optimizer.apply_gradients(zip(grads, encoder.trainable_variables + decoder.trainable_variables))

            mean_loss(loss_value)
            #mean_accuracy(to_categorical(de_outputs), outputs)

        train_loss_results.append(mean_loss.result())
        #train_accuracy_results.append(mean_accuracy.result())
        
        
        
        valid_mean_loss = tf.keras.metrics.Mean()
        
        for en, de in valid_dataset:
            de_inputs, de_outputs = decoder_data(de)
            
            hidden_states, cell_states = encoder(en)
            outputs, hidden_state_de, cell_states_de = decoder(de_inputs, hidden_states, cell_states)
        
            loss_value = loss(de_outputs, outputs)
            valid_mean_loss(loss_value)
        
        valid_loss_results.append(valid_mean_loss.result())
            
        
        print("Epoch {:03d}: Loss {:.3f}, Valid loss:{:.3%}".format(epoch, 
                                                              mean_loss.result(),
                                                              valid_mean_loss.result()))
        
    return train_loss_results, valid_loss_results

In [ ]:
train_loss_results, valid_loss_results = training_(30, 
                                                   train_dataset_mapped, 
                                                   valid_dataset_mapped, 
                                                   optimizer, 
                                                   loss, 
                                                   grad)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(12, 5))

axes[0].set_xlabel("Epochs", fontsize=14)
axes[0].set_ylabel("Loss", fontsize=14)
axes[0].set_title('Loss vs epochs')
axes[0].plot(train_loss_results)

axes[1].set_title('Valid loss vs epochs')
axes[1].set_ylabel("Loss", fontsize=14)
axes[1].set_xlabel("Epochs", fontsize=14)
axes[1].plot(valid_loss_results)
plt.show()

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.

In [ ]:
random_indices = np.random.choice(len(english), size=5, replace=False)

test_english, test_german = [], []
for idx in random_indices:
    test_english.append(english[idx])
    test_german.append(padded_german_sequences[idx])
    

test_dataset = tf.data.Dataset.from_tensor_slices((test_english, test_german))
test_dataset_mapped = map_dataset(test_dataset)

In [ ]:
for en, de in test_dataset_mapped:
    de_inputs = de[:,0]
    
    hidden_states, cell_states = encoder(en)
    outputs, hidden_state_de, cell_states_de = decoder(de_inputs, hidden_states, cell_states)
    
    

In [ ]:
for idx in random_indices:
    print(f"English sentence:\t\t{english[idx]}")
    print(f"German sentence:\t\t{german[idx]}")
    print(f"German sentence sequence:\t{tokenized_german[idx]}\n")